# Data Featurization

Here, we will show some simple examples of featurizing materials composition data using so-called "composition-based feature vectors", or CBFVs. This methods represents a single chemical formula as one vector based on its constituent atoms' chemical properties (refer to the paper for more information and references).

Note that the steps shown in this notebook are intended to demonstrate the best practices associated with featurizing materials data, using *one* way of featurizing materials composition data as an example. 
Depending on your input data and your particular modeling needs, the data featurization method and procedure you use may be different than the example shown here.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

from collections import OrderedDict

#Set a random seed to ensure reproducibility across runs
RNG_SEED = 42 #the answer to ultimate question of life, the universe, and everything
np.random.seed(RNG_SEED)

## Loading data


We will start with the dataset splits that we saved from the last notebook.

In [2]:
#getting the paths for all the data
PATH = os.getcwd()
train_path = os.path.join(PATH, "../data_for_notebook_bestpractice/cp_train_byme.csv")
val_path = os.path.join(PATH, "../data_for_notebook_bestpractice/cp_val_byme.csv")
test_path = os.path.join(PATH, "../data_for_notebook_bestpractice/cp_test_byme.csv")

#now that the paths are created, we can make the dataframes
df_train = pd.read_csv(train_path)
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print("df_train dataframe shape is", df_train.shape)
print("df_val dataframe shape is", df_val.shape)
print("df_test dataframe shape is", df_test.shape)

df_train dataframe shape is (3214, 3)
df_val dataframe shape is (980, 3)
df_test dataframe shape is (370, 3)


## Sub-sampling your data (optional)

If your dataset is too large, you can subsample it to be a smaller size.
This is useful for prototyping and for making quick sanity tests of new models / parameters.

Just be aware that you do not introduce any bias into your data through the sampling.

In [3]:
df_train_sample = df_train.sample(n = 2000, random_state = RNG_SEED) #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html
df_val_sample = df_val.sample(n = 200, random_state = RNG_SEED)
df_test_sample = df_test.sample(n = 200, random_state = RNG_SEED)

print(f"shape of df_train-sample:{df_train_sample.shape}\nshape of df_val_sample:{df_val_sample.shape}\nshape of df_test_sample{df_test_sample.shape}")

shape of df_train-sample:(2000, 3)
shape of df_val_sample:(200, 3)
shape of df_test_sample(200, 3)


## Generate features using the `CBFV` package

To featurize the chemical compositions from a chemical formula (e.g. "Al2O3") into a composition-based feature vector (CBFV), we use the open-source [`CBFV` package](https://github.com/kaaiian/CBFV).

We have downloaded and saved a local copy of the package into this repository for your convenience.
For the most updated version, refer to the GitHub repository linked above.

In [4]:
#Import the package and the generate_features function 

#Import to keep track of what folder this is in!!! not calling from online, it is in the same directory as this code (kinda)
from CBFV.cbfv.composition import generate_features

The `generate_features` function from the CBFV package expects an input DataFrame containing at least the columns `['formula', 'target']`. You may also have extra feature columns (e.g., `temperature` or `pressure`, other measurement conditions, etc.).

In our dataset, `Cp` represents the target variable, and `T` is the measurement condition.
Since the `generate_features` function expects the target variable column to be named `target`, we have to rename the `Cp` column.

In [5]:
print('DataFrame column names before renaming:')
print(df_train.columns)
print(df_val.columns)
print(df_test.columns)

#renaming Cp column to "target"
rename_cp = {"Cp" : "target"} #this is a dictionary
df_train = df_train.rename(columns = rename_cp) #finds key "Cp" and replaces its value with "target"
df_val = df_val.rename(columns = rename_cp)
df_test = df_test.rename(columns = rename_cp)

df_train_sample = df_train_sample.rename(columns = rename_cp)
df_val_sample = df_val_sample.rename(columns = rename_cp)
df_test_sample = df_test_sample.rename(columns = rename_cp)

print('\nDataFrame column names after renaming:')
print(df_train.columns)
print(df_val.columns)
print(df_test.columns)

DataFrame column names before renaming:
Index(['formula', 'T', 'Cp'], dtype='object')
Index(['formula', 'T', 'Cp'], dtype='object')
Index(['formula', 'T', 'Cp'], dtype='object')

DataFrame column names after renaming:
Index(['formula', 'T', 'target'], dtype='object')
Index(['formula', 'T', 'target'], dtype='object')
Index(['formula', 'T', 'target'], dtype='object')


Now we can use the `generate_features` function to generate the CBFVs from the input data.

Note that we have specified several keyword arguments in our call to `generate_features`:
* `elem_prop='oliynyk'`
* `drop_duplicates=False`
* `extend_features=True`
* `sum_feat=True`

A short explanation for the choice of keyword arguments is below:
* The `elem_prop` parameter specifies which CBFV featurization scheme to use (there are several). For this tutorial, we have chosen to use the `oliynyk` CBFV featurization scheme.
* The `drop_duplicates` parameter specifies whether to drop duplicate formulae during featurization. In our case, we want to preserve duplicate formulae in our data (`True`), since we have multiple heat capacity measurements (performed at different temperatures) for the same compound.
* The `extend_features` parameter specifies whether to include extended features (features that are not part of `['formula', 'target']`) in the featurized data. In our case, this is our measurement temperature, and we want to include this information (`True`), since this is pertinent information for the heat capacity prediction.
* The `sum_feat` parameter specifies whether to calculate the sum features when generating the CBFVs for the chemical formulae. We do in our case (`True`).

For more information about the `generate_features` function and the CBFV featurization scheme, refer to the GitHub repository and the accompanying paper to this notebook.

# need to play around with this generate features function
# probably in a new notebook 
# just want a feel of what its inputs, outputs, etc are

In [6]:
#note that what is labelled here is unscaled!!!
X_train_unscaled, y_train, formulae_train, skipped_train = generate_features(df_train_sample, elem_prop = 'oliynyk', drop_duplicates = False, extend_features = True, sum_feat = True)
X_val_unscaled, y_val, formulae_val, skipped_val = generate_features(df_val_sample, elem_prop = 'oliynyk', drop_duplicates = False, extend_features = True, sum_feat = True)
X_test_unscaled, y_test, formulae_test, skipped_test = generate_features(df_test_sample, elem_prop = 'oliynyk', drop_duplicates = False, extend_features = True, sum_feat = True)

#outputs feature dfs for train val and test
#x involves input data (here it is temp and Cp)
#y_train is the corresponding "target" values for the X_train__unscaled
#formulae train is df of the chemical formulaes for the corresponding indexes
#eg sumatomicnumber for index 0 of x_train is 32, chemical formula is given in formula train ()

#I think drop_duplicates will remove a formula if seen more than once i.e. only keep first example of it
#for Cp there will be same formula at many different temps so we don't want to get rid of duplicates
#therefore it is set to false

#dunno what skipped_train is

Processing Input Data: 100%|████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 23517.59it/s]


	Featurizing Compositions...


Assigning Features...: 100%|████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 12668.07it/s]


	Creating Pandas Objects...


Processing Input Data: 100%|██████████████████████████████████████████████████████| 200/200 [00:00<00:00, 18182.74it/s]


...

	Creating Pandas Objects...


Processing Input Data: 100%|██████████████████████████████████████████████████████| 200/200 [00:00<00:00, 25011.50it/s]


	Featurizing Compositions...


Assigning Features...: 100%|███████████████████████████████████████████████████████| 200/200 [00:00<00:00, 9525.69it/s]

	Creating Pandas Objects...


To see what a featurized X matrix looks like, `.head()` will show us some rows:

In [7]:
X_train_unscaled.head()

,sum_Atomic_Number,sum_Atomic_Weight,sum_Period,sum_group,sum_families,sum_Metal,sum_Nonmetal,sum_Metalliod,sum_Mendeleev_Number,sum_l_quantum_number,...,range_Melting_point_(K),range_Boiling_Point_(K),range_Density_(g/mL),range_specific_heat_(J/g_K)_,range_heat_of_fusion_(kJ/mol)_,range_heat_of_vaporization_(kJ/mol)_,range_thermal_conductivity_(W/(m_K))_,range_heat_atomization(kJ/mol),range_Cohesive_energy,T
0,32.0,65.116040,8.0,30.0,15.0,1.0,2.0,0.0,162.0,2.0,...,2.642621e+06,4.742507e+06,0.858492,0.021622,2388.183171,22965.815879,3091.366423,66594.888889,7.034755,600.0
1,28.0,53.491200,9.0,36.0,43.0,0.0,6.0,0.0,544.0,2.0,...,4.363940e+03,8.544527e+03,0.000002,40.816697,1.696930,17.270367,0.006030,22037.555556,4.284089,457.7
2,46.0,98.887792,14.0,72.0,36.0,3.0,4.0,0.0,441.0,4.0,...,4.009057e+05,1.662798e+06,0.601941,1.321867,10.138486,13933.526946,6716.921700,10368.666667,1.070067,300.0
3,20.0,41.988171,5.0,18.0,9.0,1.0,1.0,0.0,95.0,1.0,...,2.521744e+04,2.868138e+05,0.234886,0.042025,1.372178,2194.463394,4968.283245,225.000000,0.018632,2800.0
4,82.0,207.200000,6.0,14.0,5.0,1.0,0.0,0.0,81.0,1.0,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1400.0


In [8]:
X_train_unscaled.shape

(2000, 177)

Note the `sum` features in the CBFV, which we have included by using `sum_feat=True` in the call to `generate_features`.

Also note the temperature column `T` at the end of this featurized data.

What we have done above is featurize the input data. In the featurized data, each row contains a unique CBFV that describes a given chemical composition.

## Data scaling & normalization

For numerical input data, scaling and normalization of the features often improves the model performance.
Scaling can partially correct the discrepancy between the orders of magnitudes of the features (e.g., some numerical features being much larger or smaller than others).
This typically improves the model learning performance, and in turn, improves the model performance.

We will scale then normalize our input data using scikit-learn's built-in `StandardScaler` class and `normalize` function.

Note, in addition to `StandardScaler`, other scalers such as `RobustScaler` and `MinMaxScaler` are also available in scikit-learn. Consult the documentation for the details and when to use them.

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

## Scaling the data

First, we instantiate the scaler object.

In a `StandardScaler` object:
* During the `fit` process, the statistics of the input data (mean and standard deviation) are computed.
* Then, during the `transform` process, the mean and standard deviation values calculated above are used to scale the data to having zero-mean and unit variance.

Therefore, for the first time usage of the scaler, we call the `.fit_transform()` method to fit the scaler to the input data, and then to transform the same data.
For subsequent uses, since we have already computed the statistics, we only call the `.transform()` method to scale data.

# **Note:** you should *only* `.fit()` the scaler using the training dataset statistics, and then use these same statistics from the training dataset to `.transform()` the other datasets (validation and train).

In [10]:
#Always scale first and then normalise!!!
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train_unscaled) #calculated mean and standard dev, scales to 0 mean and sigma = 1
X_val = scaler.transform(X_val_unscaled)         #scales the val and test, from stats found in train (scalar.fit())
X_test = scaler.transform(X_test_unscaled)

## Normalizing the scaled data

We repeat a similar process for normalizing the data.
Here, there is no need to first fit the normalizer, since the normalizer scales the rows of the input data to unit norm independently of other rows.

The normalizer is different to a Scaler in that the normalizer acts row-wise, whereas a Scaler acts column-wise on the input data.

In [11]:
#I need to look into scaling vs normalising more
#Possibly plot histogram of some data I have and see how it changes before and after scaling/normalisation
X_train = normalize(X_train)
X_val = normalize(X_val)
X_test = normalize(X_test)

# Modeling using "classical" machine learning models

Here we implement some classical ML models from `sklearn`:

* Ridge regression
* Support vector machine
* Linear support vector machine
* Random forest
* Extra trees
* Adaptive boosting
* Gradient boosting
* k-nearest neighbors
* Dummy (if you can't beat this, something is wrong.)

Note: the Dummy model types from `sklearn` act as a good sanity check for your ML studies. If your models do not perform significantly better than the equivalent Dummy models, then you should know that something has gone wrong in your model implementation.

In [12]:
from time import time

from sklearn.dummy import DummyRegressor

from sklearn.linear_model import Ridge

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.neighbors import KNeighborsRegressor

from sklearn.svm import SVR
from sklearn.svm import LinearSVR

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In addition, we define some helper functions.

In [13]:
def instantiate_model(model_name):  #model_name is callable, putting the () calls model_name
    model = model_name()       #assigns class "model_name()" to varianle "model"  
    return model               #this is because the name of a model is part of its class
                               #i.e the name of a model is an integral attribute of the model
                               #need to learn more about classes, objects, etc


def fit_model(model, X_train, y_train): #input is the model, input, and train output
    ti = time()
    model = instantiate_model(model)    #sets model to model_name(), eg if using SVR
    model.fit(X_train, y_train)         #uses model.fit() instead of SVR.fit()
    fit_time = time() - ti              #this means it can be iterated easier
    return model, fit_time              #returns model (which is the model name) and the time it took to fit it
                                        #this basically trains the model


def evaluate_model(model, X, y_act): #this gets a prediction from already trained model, then compares to actual data
    y_pred = model.predict(X)        
    r2 = r2_score(y_act, y_pred)
    mae = mean_absolute_error(y_act, y_pred)
    rmse_val = mean_squared_error(y_act, y_pred, squared=False)
    return r2, mae, rmse_val


def fit_evaluate_model(model, model_name, X_train, y_train, X_val, y_act_val): #fits (trains) model
    model, fit_time = fit_model(model, X_train, y_train)                       #then evaluates on train and val data
    r2_train, mae_train, rmse_train = evaluate_model(model, X_train, y_train) 
    r2_val, mae_val, rmse_val = evaluate_model(model, X_val, y_act_val) 
    result_dict = {                                                      
        'model_name': model_name,
        'model_name_pretty': type(model).__name__,            #creates a dictionary
        'model_params': model.get_params(),                   #this is in the form "name of thing" : "value of thing"
        'fit_time': fit_time,                                 #can then be iterated to create dataframe
        'r2_train': r2_train,                                 #then retuns ("model?") and the dictionary
        'mae_train': mae_train,                               #this is basically one row of a dataframe
        'rmse_train': rmse_train,                             
        'r2_val': r2_val,
        'mae_val': mae_val,
        'rmse_val': rmse_val}
    return model, result_dict


def append_result_df(df, result_dict):  #creates df from dictionary, then returns it
    df_result_appended = df.append(result_dict, ignore_index = True) #ignore index means rows not numbered
    return df_result_appended                #basically adds the evaluation from a model onto this df when iterating
                                             #important that you don't keep appendening to the same df with data you need 
                                             #already in it ***



def append_model_dict(dic, model_name, model): #creates list of model names
    dic[model_name] = model                    #with the way its set up each input is a tuple as it come from a dictionary
    return dic

Build an empty DataFrame to store model results:

In [14]:
#creating a data frame and labelling columns
df_classics = pd.DataFrame(columns=['model_name',
                                    'model_name_pretty',
                                    'model_params',
                                    'fit_time',
                                    'r2_train',
                                    'mae_train',
                                    'rmse_train',
                                    'r2_val',
                                    'mae_val',
                                    'rmse_val'])
df_classics

,model_name,model_name_pretty,model_params,fit_time,r2_train,mae_train,rmse_train,r2_val,mae_val,rmse_val


## Define the models

Here, we instantiate several classical machine learning models for use.
For demonstration purposes, we instantiate the models with their default model parameters.

Some of the models listed above can perform either regression or classification tasks.
Because our ML task is a regression task (prediction of the continuous-valued target, heat capacity), we choose the regression variant of these models.

Note: the `DummyRegressor()` instance acts as a good sanity check for your ML studies. If your models do not perform significantly better than the `DummyRegressor()`, then you know something has gone awry.

In [15]:
#Build a dictionary of model names
classic_model_names = OrderedDict({ #OrderedDict preserves the order in which the keys are inserted 
    'dumr': DummyRegressor,         #A regular dict doesn’t track the insertion order and iterating
    'rr': Ridge,                    # it gives the values in an arbitrary order
    'abr': AdaBoostRegressor,
    'gbr': GradientBoostingRegressor,
    'rfr': RandomForestRegressor,
    'etr': ExtraTreesRegressor,
    'svr': SVR,
    'lsvr': LinearSVR,
    'knr': KNeighborsRegressor,
})

classic_model_names.items()

odict_items([('dumr', <class 'sklearn.dummy.DummyRegressor'>), ('rr', <class 'sklearn.linear_model._ridge.Ridge'>), ('abr', <class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>), ('gbr', <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>), ('rfr', <class 'sklearn.ensemble._forest.RandomForestRegressor'>), ('etr', <class 'sklearn.ensemble._forest.ExtraTreesRegressor'>), ('svr', <class 'sklearn.svm._classes.SVR'>), ('lsvr', <class 'sklearn.svm._classes.LinearSVR'>), ('knr', <class 'sklearn.neighbors._regression.KNeighborsRegressor'>)])

## Instantiate and fit the models

Now, we can fit the ML models.

We will loop through each of the models listed above. For each of the models, we will:
* instantiate the model (with default parameters)
* fit the model using the training data
* use the fitted model to generate predictions from the validation data
* evaluate the performance of the model using the predictions
* store the results in a DataFrame for analysis

Note: this may take several minutes, depending on your hardware/software environment, dataset size and featurization scheme (CBFV).

In [16]:
#***here I got confused because shape of df was bigger than should've been, this is because I ran
#this block of code multiple times without cleaning df_classics

#Instantiate a dictionary to store the model objects
classic_models = OrderedDict()

# Keep track of elapsed time
ti = time()

#google for x,y in python if looking back at this
#creating a loop to fit each model and then store the results
#.items() returns dictionary to tuple
#in this specific case it returns it as(model_name, model)
#this means you are iterating through tuples not integers or something
#so the in between "for" and "in" you need both the model_name, model to be placed there
for model_name, model in classic_model_names.items(): 
   
    print("Now fitting model", model_name, ":", model.__name__)
    
    model, result_dict = fit_evaluate_model(model, model_name, X_train, y_train, X_val, y_val) #defined above
    #this gives what model is being used and returns dictionary of results
    
    df_classics = append_result_df(df_classics, result_dict) #add results_dict to  df_classics
    #print(f"\n\ndf_classics is\n{df_classics}\n\n")
    
    classic_models = append_model_dict(classic_models, model_name, model) #not sure
    #print(f"classic_models is {classic_models}\n")
    
dt = time() - ti

print("\nFinished fitting", len(classic_models), "\n", dt, "seconds taken") #using the f sting thing is probably better idk


df_classics.shape

Now fitting model dumr : DummyRegressor
Now fitting model rr : Ridge
Now fitting model abr : AdaBoostRegressor
Now fitting model gbr : GradientBoostingRegressor
Now fitting model rfr : RandomForestRegressor
Now fitting model etr : ExtraTreesRegressor
Now fitting model svr : SVR
Now fitting model lsvr : LinearSVR
Now fitting model knr : KNeighborsRegressor

Finished fitting 9 
 27.241197109222412 seconds taken


(9, 10)

Now, we can look at the results.

You will notice, that some of the models (such as RandomForestRegressor, ExtraTreesRegressor and GradientBoostingRegressor) have completely memorized the training data, as evidenced by the very high r2_train scores of ~1.0.

In [ ]:
#Sort in order of increasing validation r2 score
df_classics = df_classics.sort_values('r2_val', ignore_index = True) #don't want rows labelled so set to true
df_classics                                                          #although nothing seems to change when it's False?

You can now also access the full details of the models by inspecting the `classic_models` dictionary that we populated.

In [ ]:
classic_models

## Evaluating model performance on validation dataset

Now comes the time to evaluate the trained models on the validation set.

Remember, we use the same validation set to evaluate all models. This ensures a fair comparison.

In addition, we plot the predicted vs. actual plots using the predictions made by each trained model on the same validation set.

In [ ]:
#https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D.set_markeredgecolor

#input is actual values, predicted values, model used, regline is set to true automatically, label is a string
def plot_pred_act(actual, predicted, model, reg_line = True, label = ""):
    xy_max = np.max([np.max(actual), np.max(predicted)])
    
    plot = plt.figure(figsize = (6,6)) #plt.figure() makes an object explicitly, it can be maipulated then later
    plt.plot(actual, predicted, 'o', ms = 9, mec = 'k', mfc = 'pink', alpha = 0.4) 
        #o is circle marker, msis markersize, mec is marker edge colour, mfc is marker face colour                          
        #alpha is opacity
    plt.plot([0, xy_max], [0, xy_max], 'k--', label='ideal') #creates ideal line from bleft to tright of square
    
    #below gives regression line for the predicted data
    if reg_line: #shorthand for "if reg_line == True:"
        polyfit = np.polyfit(actual, predicted, deg = 1) #least squares polynomial fit, deg = 1 for linear line
                                                 #returns [x coeffcient     y intercept]
        #np.poly1d([1,2,3]) == (x-1)(x-2)(x-3)
        #not sure why finding unique values, presume it's so not several markers for same point
        reg_ys = np.poly1d(polyfit)(np.unique(actual)) #DONT UNDERSTAND THIS ONE
        plt.plot(np.unique(actual), reg_ys, alpha=0.8, label='linear fit') #plots reg line
     

    plt.axis('scaled')
    plt.xlabel("Actual" + label) #label is a string so need to use + not ,
    plt.ylabel("Predicted" + label)
    
    plt.title(f'{type(model).__name__}, r2: {r2_score(actual, predicted)}')
    #plt.title(type(model).__name__ ,"r2:", r2_score(actual,predicted)) #can' get this to not give errors
    
    plt.legend(loc = "best")
    
    #screwing around with background colours
    #ax = plt.gca()
    #ax.set_facecolor("yellow")
    
    return plot

In [ ]:
#row goes from 0 to no. of rows in df_classics
for row in range(df_classics.shape[0]):
    model_name = df_classics.iloc[row]['model_name'] #using iloc[] because row is an integer not a label

    model = classic_models[model_name]
    y_act_val = y_val
    y_pred_val = model.predict(X_val)

    plot = plot_pred_act(y_act_val, y_pred_val, model, label='$\mathrm{C}_\mathrm{p}$ (J / mol K)') #dunno about the label fam

## Re-training the best-performing model on combined train + validation dataset

After you have finalized your model, you can re-train your model (using the same hyperparameters) again on the combined train + validation datasets, and finally, evaluate your model on the held-out test dataset.

By training on the combined train + validation dataset after you have finished tuning your model, you give it more training data, which should lead to an increase in the model performance.

In [ ]:
#manually getting the best-performing model that we have tested
best_row = df_classics.iloc[-1, :].copy()

#Get the model type and model parameters
model_name = best_row['model_name']
model_params = best_row['model_params']

#Instantiate the model again using the parameters
#putting (**model_params) replaces default parameters with ones already used I THINK DONT ACTUALLY KNOW
model = classic_model_names[model_name](**model_params)
print(model)

In [ ]:
#Concatenate the train and validation datasets together
X_train_new = np.concatenate((X_train, X_val), axis = 0) #these are arrays so np.concatenate()
y_train_new = pd.concat((y_train, y_val), axis = 0)      #these are dataframes so pd.concat()

print(X_train_new.shape)
print(y_train_new.shape)

Finally, we can fit the model on the combined train + validation dataset.

In [ ]:
ti = time()

model.fit(X_train_new, y_train_new)

dt = time() - ti 

print("Finished fitting best model, total time:", dt)

## Testing the re-trained model on the test dataset

After re-fitting the best model on the train+validation dataset, you can finally test it on the test dataset.
**Remember:** you should only do this *once!*

In [ ]:
#maybe need to be a bit more clear on where y test, ypredtest, etc come from and what happens to test inputs

y_act_test = y_test
y_pred_test = model.predict(X_test)

r2, mae, rmse = evaluate_model(model, X_test, y_test)
print("r2:", r2)
print("mae", mae)
print("rmse", rmse)

plot = plot_pred_act(y_act_test, y_pred_test, model, label='$\mathrm{C}_\mathrm{p}$ (J / mol K)')

We see that our model achieves decent performance on the held-out test dataset.

# Effect of train/validation/test dataset split

Using different train/validation/test splits can dramatically affect your model performance, even for classical ML models.

Here, we provide a little demonstration.

In [ ]:
#generating same features as above, just doing it again as data has been manipulated
X_train_unscaled, y_train, formulae_train, skipped_train = generate_features(df_train, elem_prop='oliynyk', drop_duplicates=False, extend_features=True, sum_feat=True)
X_val_unscaled, y_val, formulae_val, skipped_val = generate_features(df_val, elem_prop='oliynyk', drop_duplicates=False, extend_features=True, sum_feat=True)
X_test_unscaled, y_test, formulae_test, skipped_test = generate_features(df_test, elem_prop='oliynyk', drop_duplicates=False, extend_features=True, sum_feat=True)

In [ ]:
X_train_original = X_train_unscaled.copy()
X_val = X_val_unscaled.copy()
X_test = X_test_unscaled.copy()

y_train_original = y_train.copy()

We sample the training data using 10 random seeds, by using the `DataFrame.sample()` method with seeds ranging from 0 to 9.
We then fit 10 models, each on one of the random splits, and evaluate their performance on the same validation dataset.

In [ ]:
splits = range(10)
df_splits = pd.DataFrame(columns=['split',
                                  'r2_train',
                                  'mae_train',
                                  'rmse_train',
                                  'r2_val',
                                  'mae_val',
                                  'rmse_val'])

for split_no in splits:
    print("Fitting and evaluating random split", split_no)
    #now I need to sample the data, standerise, normalise, fit and evaluate
    
    #sampling
    X_train = X_train_original.sample(frac = 0.7, random_state = split_no) #random_state is seed for pseudorandom numbers
    y_train = y_train_original[X_train.index] #this finds the corresponding y value at the correct index (same index as X)
    
    scaler = StandardScaler()
    X_train = normalize(scaler.fit_transform(X_train))  #normalising and scaling in the one line
    X_val = normalize(scaler.transform(X_val_unscaled)) #note just .transform() here, not .fit()
    X_test = normalize(scaler.transform(X_test_unscaled))
    
    model = AdaBoostRegressor() #dunno why using this one, might mess around with this one
    model.fit(X_train, y_train) #training model on this split
    y_act_val = y_val
    y_pred_val = model.predict(X_val) #now predicting the trained model on the validation data
    
    r2_train, mae_train, rmse_train = evaluate_model(model, X_train, y_train)
    r2_val, mae_val, rmse_val = evaluate_model(model, X_val, y_val)
    result_dict = {
        'split': split_no,
        'r2_train': r2_train,
        'mae_train': mae_train,
        'rmse_train': rmse_train,
        'r2_val': r2_val,
        'mae_val': mae_val,
        'rmse_val': rmse_val}
    
    df_splits = append_result_df(df_splits, result_dict)

In [ ]:
df_splits['split'] = df_splits['split'].astype(int) #split numbers are floats instead of ints, so turning them into ints
df_splits

We then plot the train and validation $r^2$ scores for each of the 10 models.

Note the high variability in the r2_val score. In contrast, the variability in the r2_train score is comparatively lower.

In [ ]:
#dataframe.plot makes plots of Series or DataFrame
df_splits.plot('split', ['r2_train', 'r2_val'], kind = 'bar') 
#first x values. In this case it is split number
#['r2_train', 'r2_val'] are the y values for each x
#kind is bar, line, boxplot, etc
plt.title(f'Performance of {type(model).__name__}\nwith {len(splits)} different data splits') #not arsed rewriting this
plt.ylim((0.5, 1.0))
plt.ylabel('$r^2$')
plt.xlabel('Split #')
plt.legend(loc = 'lower right', framealpha = 0.9)
plt.show()

This effect is even more pronounced when we plot the mean abolute error (MAE).

In [ ]:
#same as above just mae not r2
df_splits.plot('split', ['mae_train', 'mae_val'], kind='bar')
plt.title(f'Performance of {type(model).__name__}\nwith {len(splits)} different data splits')
plt.ylabel('MAE in $\mathrm{C}_\mathrm{p}$ (J / mol K)')
plt.xlabel('Split #')
plt.legend(loc='lower right', framealpha=0.9)
plt.show()

Therefore, typically the average value of all the scores are reported, as this gives a much more accurate estimate of how well the model actually performs.

In [ ]:
avg_r2_val = df_splits['r2_val'].mean()
avg_mae_val = df_splits['mae_val'].mean()

print("Average validation r2 is", avg_r2_val)
print("Average validation mae is", avg_r2_val)